<a href="https://colab.research.google.com/github/L3G5/volatility-bybit/blob/main/volatility_in_previous_n_minutes_BYBIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A tool to help in searching for the most volatile coins on Binance

It gets all 5-minute candles for all Binance futures pairs, gets all highest $h$ and lowest $l$ prices for last n minutes and calculates for them volatility $v = \frac{h - l}{l}$; it also calculates position $p$ of the current price $c$ between these high and low  $ p = \frac{c-l}{h-l}$, so that $p = 0$ when asset is close to the low and $p = 1$ when it is close to high.

Then it shows a table which can be sorted by clicking on a column and a colored table sorted by 1 hour column, where the blue color shows volatility and gradient from green to red shows position of thr current price relative to certain high/lows.

If you are planning to use this code a lot, get your Binance API key (with only read settings for extra privacy) [here](https://www.binance.com/en/support/faq/360002502072), copy this notebook and run it yourself so that I don't get banned for using too much data with my own API key :kekw: 

# More detailed instructions:
- Purpose of the script and what the tables show:
It is supposed to show most volatile coins on Binance Futures (to help in viewing the whole Futures market in a very short time; in choosing most volatile coins to avoid loss of opportunity, when one doesn't want to hold a coin for too long; something else?). Most condensed info is in the colorful table in the very bottom. Table contains columns "period", "period_p"; in 'period' columns there are volatility numbers for the last period, the bigger they are - the bigger is the volatility and the whole table is currently sorted by volatility in the last 2 days; in 'period_p' column there are positions of current price relative to high and low in the corresponding period, when value is near 0 it means that asset is near its low during the last period, when value is 1 - coin is near its high. The color of the cells 'period' is most blue for those cells which have the biggest volatility value among all coins on the Futures; the color of the cells 'period_p' is near red when value is near 0 and green when value is near 1. You can also use the first, uncolored, table for quick sorting by a certain column: simply click on the heading of that column. The last run of the script is shown on the top of colorful table, and you can run it yourself by clicking Runtime -> Run all


- Definitions that were used: 
1) volatility during period: 
volatility[period] = (high[timeframe]-low[period])/(low[[period]) , 
where high[period]  is the highest price during period
low[period]  is the lowest price during period
2) position of current price, current, relative to high[period] and low[period]:
pos[period] = (current - low[period])/(high[period]-low[period])

- How to modify the script:
It is easy to modify the periods: just add your number of minutes/5 to the lines to the square brackets in the line 
`for min in [1, 3, 6, 12, 24, 48, 144]:`. For example, if you want to see 20=4*5 mins volatility, you should modify the line to `for min in [1, 3, 4, 6, 12, 24, 48, 144]:`. It will only process periods up to 2 days, so if you want to see, say, 6D volatility, ypu should instead modify the line   `for day in [1, 2, 3, 5, 10, 15, 30, 60, 90]:` to   `for day in [1, 2, 3, 5, 6, 10, 15, 30, 60, 90]:`.
It is also simple to modify the column by which the colorful table is sorted: you should write the names of column which you want
to sort by in the line `df = pd.DataFrame(volatility).set_index('symbol').join(pd.DataFrame(volatility_daily).set_index('symbol')).sort_values(['90D'], ascending = False)` in the square brackets of `.sort_values(['90D']`. For example, if you want to sort by 60m_p, change this line to `df = pd.DataFrame(volatility).set_index('symbol').join(pd.DataFrame(volatility_daily).set_index('symbol')).sort_values(['60m_p'], ascending = False)`  
  
If you want to make some big changes - you can simply copy the script to your Google Colab and use your own Binance API https://www.binance.com/en/binance-api keys

In [ ]:
!pip install python-binance

import requests
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import datetime
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import datetime as dt

from binance.client import Client

from google.colab import data_table
from google.colab.data_table import DataTable
DataTable.max_columns = 36

data_table.enable_dataframe_formatter()
data_table._DEFAULT_FORMATTERS[float] = lambda x: f"{x:.3f}"

!pip install pybit

from pybit import usdt_perpetual

! pip install tradingview_ta
from tradingview_ta import *


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Download Bybit data

In [ ]:
session_unauth = usdt_perpetual.HTTP(endpoint="https://api.bybit.com")
all_symbols = pd.json_normalize(session_unauth.query_symbol()['result'])
coin_names = all_symbols.loc[all_symbols['name'].str[-1:]=='T', 'name'].to_list()

dfs = {}
dfs_daily = {}
volatility_daily = []
volatility = []
for coin in coin_names:
  tickers = session_unauth.query_kline(
    symbol=coin,
    interval=5,
    limit=144,
    from_time=int(dt.datetime.now().timestamp()-60*144)
)['result']
  tickers_daily = session_unauth.query_kline(
    symbol=coin,
    interval='D',
    limit=200,
    from_time=int(dt.datetime.now().timestamp()-60*60*24*200)
)['result']
  df = pd.DataFrame(tickers, columns = ['open_time', 'volume', 'open', 'high', 'low', 'close', 'turnover'])
  df['open_time'] = pd.to_datetime(df['open_time'], unit = 's')
  dfs[coin] = df

  df_daily = pd.DataFrame(tickers_daily, columns = ['open_time', 'volume', 'open', 'high', 'low', 'close', 'turnover'])
  df_daily['open_time'] = pd.to_datetime(df_daily['open_time'], unit = 's')
  dfs_daily[coin] = df

  last_price =  df.iloc[-1:]['close'].to_list()[0]
  volatility_row = {'symbol': coin}
  volatility_row_daily = {'symbol': coin}
  volatility_row['last_price'] = last_price
  for min in [1, 3, 6, 12, 24, 48, 144]:
    local_high = df.iloc[-min:]['high'].max()
    local_low = df.iloc[-min:]['low'].min()
    volatility_row[str(min*5)+'m'] = local_high/local_low - 1
    try:
      volatility_row[str(min*5) + 'm_p'] = (last_price - local_low)/(local_high - local_low)
    except:
      try:
        volatility_row[str(min*5) + 'm_p'] = (last_price - df.iloc[-min-1:]['low'].min())/(df.iloc[-min-1:]['high'].max() - df.iloc[-min-1:]['low'].min())
      except:
        continue

  for day in [1, 2, 3, 5, 10, 15, 30, 60, 90, 200]:
    local_high = df_daily.iloc[-day:]['high'].max()
    local_low = df_daily.iloc[-day:]['low'].min()
    volatility_row_daily[str(day)+'D'] = local_high/local_low - 1
    try:
      volatility_row_daily[str(day)+'D_p'] = (last_price - local_low)/(local_high - local_low)
    except:
      try:
        volatility_row_daily[str(day)+'D_p'] = (last_price - df.iloc[-min-1:]['low'].min())/(df.iloc[-min-1:]['high'].max() - df.iloc[-min-1:]['low'].min())
      except:
        continue
  volatility_daily.append(volatility_row_daily)
  volatility.append(volatility_row)
  
print('Last update is:', dt.datetime.now(),  ' UTC')


<ipython-input-22-8b1c25088313>:39: RuntimeWarning: invalid value encountered in double_scalars
  volatility_row[str(min*5) + 'm_p'] = (last_price - local_low)/(local_high - local_low)
<ipython-input-22-8b1c25088313>:39: RuntimeWarning: invalid value encountered in long_scalars
  volatility_row[str(min*5) + 'm_p'] = (last_price - local_low)/(local_high - local_low)


Last update is: 2023-01-21 13:01:04.751640  UTC


In [ ]:
tas_list = []
pss_week = get_multiple_analysis(screener="crypto", interval=Interval.INTERVAL_1_WEEK, symbols=["BYBIT:"+i+".P" for i in coin_names])
pss_day = get_multiple_analysis(screener="crypto", interval=Interval.INTERVAL_1_DAY, symbols=["BYBIT:"+i+".P" for i in coin_names])
pss_4hours = get_multiple_analysis(screener="crypto", interval=Interval.INTERVAL_4_HOURS , symbols=["BYBIT:"+i+".P" for i in coin_names])
pss_1hour = get_multiple_analysis(screener="crypto", interval=Interval.INTERVAL_1_HOUR , symbols=["BYBIT:"+i+".P" for i in coin_names])

for coin in ["BYBIT:"+i+".P" for i in coin_names]:
  try: 
    pss_week_ = pss_week[coin].indicators
  except:
    pss_week_ = {}
  try: 
    pss_day_ = pss_day[coin].indicators
  except:
    pss_day_ = {}
  try: 
    pss_4hours_ = pss_4hours[coin].indicators
  except:
    pss_4hours_ = {}
  try: 
    pss_1hour_ = pss_1hour[coin].indicators
  except:
    pss_1hour_ = {}
  tas_list.append([coin[6:-2], pss_week_.get('SMA200'), pss_day_.get('SMA200'), pss_4hours_.get('SMA100'), pss_1hour_.get('SMA100'), pss_day_.get('RSI'),  
                   pss_4hours_.get('RSI'), pss_1hour_.get('RSI')])
technicals = pd.DataFrame(tas_list, columns = ['symbol', 'SMA200W', 'SMA200D', 'SMA400H', 'SMA100H', 'RSID', 'RSI4H', 'RSIH']).set_index('symbol')

In [ ]:
def style_bigger(v, border, props=''):
    return props if v > border else None
def style_smaller(v, border, props=''):
    return props if v < border else None

df = technicals.join(pd.DataFrame(volatility).set_index('symbol')).join(pd.DataFrame(volatility_daily).set_index('symbol')).sort_values(['3D'], ascending = False)
for sma_ in ['SMA200W',	'SMA200D',	'SMA400H',	'SMA100H']:
  df[sma_] = df['last_price']/df[sma_]
df = df.drop(columns = ['last_price'])
# df_styled = df.style.background_gradient(subset=['5m']).format(precision=2)
# df_styled = df.style.background_gradient(subset=['5m']).format(precision=3)
df_styled = df.style.format(precision=3)
for col in df.columns[:]:
  if col[0] == 'S':
    df_styled = df_styled.highlight_between(left=0.9, right=1.1, axis=1, props='color:white; background-color:purple;', subset=[col])
  elif col[0] == 'R':
    df_styled = df_styled.applymap(style_bigger, border = 70, props='color:red;', subset=[col])
    df_styled = df_styled.applymap(style_smaller, border = 0.3, props='color:green;', subset=[col])
  elif col[-1] != 'p':
    df_styled = df_styled.background_gradient(cmap = 'Blues', subset=[col])
  else:
    df_styled = df_styled.background_gradient(cmap = 'RdYlGn', subset=[col])
df_styled.set_sticky(axis=0).set_sticky(axis=1)

In [ ]:
technicals

,SMA200W,SMA200D,SMA400H,SMA100H,RSID,RSI4H,RSIH
symbol,,,,,,,
10000NFTUSDT,None,0.005463,0.004364,0.004508,59.955354,63.289070,65.669622
1000BONKUSDT,None,NaN,NaN,0.001231,36.479282,55.197379,54.419212
1000BTTUSDT,None,0.000790,0.000685,0.000731,75.446504,66.774793,65.191150
1000LUNCUSDT,None,NaN,0.167557,0.172587,59.171628,59.124889,57.306499
1000XECUSDT,None,0.035955,0.027293,0.029501,67.428325,59.560500,60.134440
...,...,...,...,...,...,...,...
YGGUSDT,None,0.404305,0.212130,0.231060,64.548272,61.059621,61.307176
ZECUSDT,None,53.034750,43.788000,45.079500,65.318859,70.555103,63.752452
ZENUSDT,None,13.319250,9.501000,10.226000,65.077521,62.832790,64.254242


In [ ]:
data_table.DataTable(df, max_columns=50, max_rows=200, num_rows_per_page=25, )


,SMA200W,SMA200D,SMA400H,SMA100H,RSID,RSI4H,RSIH,5m,5m_p,15m,...,15D,15D_p,30D,30D_p,60D,60D_p,90D,90D_p,200D,200D_p
symbol,,,,,,,,,,,,,,,,,,,,,
APTUSDT,NaN,NaN,1.963068,1.462545,90.369778,82.840025,80.050976,0.011628,0.586207,0.021044,...,2.504021,0.944861,3.271242,0.948551,3.271242,0.948551,3.271242,0.948551,3.271242,0.948551
OPUSDT,NaN,2.209598,1.597857,1.293790,86.825003,80.389439,71.601539,0.002314,0.181818,0.008433,...,1.341647,0.922483,1.827273,0.931281,1.976077,0.933111,2.273684,0.936053,5.173697,0.947002
API3USDT,NaN,1.092776,1.392209,1.323058,76.554325,84.702714,91.642206,0.010101,0.000000,0.018049,...,0.681529,0.911883,0.862903,0.922897,0.862903,0.922897,1.514113,0.525965,1.905242,0.417989
MAGICUSDT,NaN,NaN,1.436351,1.178432,67.812861,69.874068,58.424337,0.004935,0.000000,0.007569,...,1.170009,0.807657,1.245695,0.813043,1.245695,0.813043,1.245695,0.813043,1.245695,0.813043
HBARUSDT,NaN,1.242295,1.469076,1.203382,87.408152,81.704025,62.352876,0.000000,NaN,0.007666,...,1.090791,0.799049,1.286525,0.813671,1.286525,0.813671,1.286525,0.813671,1.394326,0.750763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10000NFTUSDT,NaN,0.851176,1.065548,1.031545,59.955354,63.289070,65.669622,0.002153,0.500000,0.011957,...,0.175743,0.859155,0.187500,0.866667,0.245000,0.663265,0.667500,0.243446,0.751250,0.216306
SUNUSDT,NaN,1.022884,1.088779,1.024867,69.523566,62.006466,60.947285,0.000000,NaN,0.000847,...,0.225025,0.856502,0.247688,0.867220,0.462963,0.466667,0.595445,0.473588,1.034707,0.272537
1000BTTUSDT,NaN,0.957059,1.104149,1.034554,75.446504,66.774793,65.191150,0.002653,1.000000,0.002653,...,0.208931,0.984733,0.234528,0.986111,0.234528,0.986111,0.381107,0.606838,0.744300,0.310722


In [ ]:
data_table.DataTable(df[df.columns[9:]], max_columns=50, max_rows=200, num_rows_per_page=25, )


,15m,15m_p,30m,30m_p,60m,60m_p,120m,120m_p,240m,240m_p,...,15D,15D_p,30D,30D_p,60D,60D_p,90D,90D_p,200D,200D_p
symbol,,,,,,,,,,,,,,,,,,,,,
APTUSDT,0.021044,0.769231,0.023945,0.796610,0.044288,0.887850,0.048628,0.897436,0.050812,0.901639,...,2.504021,0.944861,3.271242,0.948551,3.271242,0.948551,3.271242,0.948551,3.271242,0.948551
OPUSDT,0.008433,0.300000,0.016146,0.631579,0.043552,0.234146,0.081504,0.410667,0.081504,0.410667,...,1.341647,0.922483,1.827273,0.931281,1.976077,0.933111,2.273684,0.936053,5.173697,0.947002
API3USDT,0.018049,0.281250,0.032018,0.589286,0.041546,0.680556,0.069444,0.450000,0.071304,0.463415,...,0.681529,0.911883,0.862903,0.922897,0.862903,0.922897,1.514113,0.525965,1.905242,0.417989
MAGICUSDT,0.007569,0.028986,0.016352,0.040268,0.016352,0.040268,0.038112,0.579412,0.038112,0.579412,...,1.170009,0.807657,1.245695,0.813043,1.245695,0.813043,1.245695,0.813043,1.245695,0.813043
HBARUSDT,0.007666,0.727273,0.011896,0.823529,0.018195,0.538462,0.023207,0.636364,0.030453,0.720930,...,1.090791,0.799049,1.286525,0.813671,1.286525,0.813671,1.286525,0.813671,1.394326,0.750763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10000NFTUSDT,0.011957,0.909091,0.011957,0.909091,0.016376,0.933333,0.024202,0.954545,0.024202,0.954545,...,0.175743,0.859155,0.187500,0.866667,0.245000,0.663265,0.667500,0.243446,0.751250,0.216306
SUNUSDT,0.000847,1.000000,0.000847,1.000000,0.003396,1.000000,0.007673,1.000000,0.008532,1.000000,...,0.225025,0.856502,0.247688,0.867220,0.462963,0.466667,0.595445,0.473588,1.034707,0.272537
1000BTTUSDT,0.002653,1.000000,0.002653,1.000000,0.010695,1.000000,0.013405,1.000000,0.013405,1.000000,...,0.208931,0.984733,0.234528,0.986111,0.234528,0.986111,0.381107,0.606838,0.744300,0.310722
